In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [3]:
import datetime as dt
import pandas as pd

df_covid19 = pd.read_json('https://pomber.github.io/covid19/timeseries.json')
df_covid19.head()
countries= df_covid19.columns.to_list()

def get_countries(countries):
    df_world = pd.DataFrame({dt.datetime:None, int:None, int:None, object:None}, 
                            columns = ['date','confirmed','deaths','recovered', 'country'])

    for country in countries:
      df_country = pd.json_normalize(df_covid19[str(country)])
      df_country['country'] = country
      df_world = df_world.append(df_country, ignore_index = True)

    df_world['date'] =  pd.to_datetime(df_world['date'] , infer_datetime_format=True)

    return df_world

df_world = get_countries(countries)
df_world.to_csv("covid19_200508.csv")

In [4]:
df = spark.read.csv("covid19_200508.csv")

In [5]:
df.count()

65896

In [6]:
df.describe()

DataFrame[summary: string, _c0: string, _c1: string, _c2: string, _c3: string, _c4: string, _c5: string]

In [7]:
df = spark.read.format("csv").option("header", "true").load("covid19_200508.csv")
df.describe()

DataFrame[summary: string, _c0: string, date: string, confirmed: string, deaths: string, recovered: string, country: string]

In [8]:
df.createOrReplaceTempView("covid19")

In [9]:
def execute_query(query, rows=20):
    sqlDF = spark.sql(query)
    sqlDF.show(rows)

In [10]:
execute_query("SELECT * FROM covid19 where country= 'Mexico' and date >= '2020-05-01'",40) 

+-----+----------+---------+------+---------+-------+
|  _c0|      date|confirmed|deaths|recovered|country|
+-----+----------+---------+------+---------+-------+
|39430|2020-05-01|    20739|  1972|    12377| Mexico|
|39431|2020-05-02|    22088|  2061|    12377| Mexico|
|39432|2020-05-03|    23471|  2154|    13447| Mexico|
|39433|2020-05-04|    24905|  2271|    13447| Mexico|
|39434|2020-05-05|    26025|  2507|    16810| Mexico|
|39435|2020-05-06|    27634|  2704|    17781| Mexico|
|39436|2020-05-07|    29616|  2961|    17781| Mexico|
|39437|2020-05-08|    31522|  3160|    20314| Mexico|
|39438|2020-05-09|    33460|  3353|    21824| Mexico|
|39439|2020-05-10|    35022|  3465|    21824| Mexico|
|39440|2020-05-11|    36327|  3573|    23100| Mexico|
|39441|2020-05-12|    38324|  3926|    25935| Mexico|
|39442|2020-05-13|    40186|  4220|    26990| Mexico|
|39443|2020-05-14|    42595|  4477|    28475| Mexico|
|39444|2020-05-15|    45032|  4767|    30451| Mexico|
|39445|2020-05-16|    47144|

In [11]:
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- date: string (nullable = true)
 |-- confirmed: string (nullable = true)
 |-- deaths: string (nullable = true)
 |-- recovered: string (nullable = true)
 |-- country: string (nullable = true)



In [15]:
df = df.withColumnRenamed('_c0', 'item')

In [17]:
from pyspark.sql import functions as F
from pyspark.sql.types import *

df = df.withColumn('item', F.col('item').cast("int"))
df = df.withColumn('date', F.col('date').cast("date"))
df = df.withColumn('confirmed', F.col('confirmed').cast("int"))
df = df.withColumn('deaths', F.col('deaths').cast("int"))
df = df.withColumn('recovered', F.col('recovered').cast("int"))
df = df.withColumn('country', F.col('country').cast("string"))

df.printSchema()

root
 |-- item: integer (nullable = true)
 |-- date: date (nullable = true)
 |-- confirmed: integer (nullable = true)
 |-- deaths: integer (nullable = true)
 |-- recovered: integer (nullable = true)
 |-- country: string (nullable = true)



In [18]:
df.schema["item"].nullable = False

In [19]:
df.createOrReplaceTempView("covid19")

In [20]:
execute_query("""
              SELECT a.*, 
                  date_add(date, 1) as date_1,
                  date_sub(date, 1) as date_1
              FROM covid19 a where a.country= 'Mexico' and date >= '2020-05-01'
              """,40)

+-----+----------+---------+------+---------+-------+----------+----------+
| item|      date|confirmed|deaths|recovered|country|    date_1|    date_1|
+-----+----------+---------+------+---------+-------+----------+----------+
|39430|2020-05-01|    20739|  1972|    12377| Mexico|2020-05-02|2020-04-30|
|39431|2020-05-02|    22088|  2061|    12377| Mexico|2020-05-03|2020-05-01|
|39432|2020-05-03|    23471|  2154|    13447| Mexico|2020-05-04|2020-05-02|
|39433|2020-05-04|    24905|  2271|    13447| Mexico|2020-05-05|2020-05-03|
|39434|2020-05-05|    26025|  2507|    16810| Mexico|2020-05-06|2020-05-04|
|39435|2020-05-06|    27634|  2704|    17781| Mexico|2020-05-07|2020-05-05|
|39436|2020-05-07|    29616|  2961|    17781| Mexico|2020-05-08|2020-05-06|
|39437|2020-05-08|    31522|  3160|    20314| Mexico|2020-05-09|2020-05-07|
|39438|2020-05-09|    33460|  3353|    21824| Mexico|2020-05-10|2020-05-08|
|39439|2020-05-10|    35022|  3465|    21824| Mexico|2020-05-11|2020-05-09|
|39440|2020-

In [21]:
execute_query("""
              SELECT 
                  a.item as item_a,
                  b.item as item_b,
                  a.confirmed as confirmed_a, 
                  a.deaths as deaths_a, 
                  a.recovered as recovered_a,
                  b.confirmed as confirmed_b, 
                  b.deaths as deaths_b, 
                  b.recovered as recovered_b,
                  a.date as date_a,
                  b.date as date_b
              FROM covid19 a 
              inner join covid19 b on a.country = b.country and a.date = date_add(b.date, 1)
              where a.country= 'Mexico' and a.date >= '2020-05-01'
              """,40)

+------+------+-----------+--------+-----------+-----------+--------+-----------+----------+----------+
|item_a|item_b|confirmed_a|deaths_a|recovered_a|confirmed_b|deaths_b|recovered_b|    date_a|    date_b|
+------+------+-----------+--------+-----------+-----------+--------+-----------+----------+----------+
| 39430| 39429|      20739|    1972|      12377|      19224|    1859|      11423|2020-05-01|2020-04-30|
| 39431| 39430|      22088|    2061|      12377|      20739|    1972|      12377|2020-05-02|2020-05-01|
| 39432| 39431|      23471|    2154|      13447|      22088|    2061|      12377|2020-05-03|2020-05-02|
| 39433| 39432|      24905|    2271|      13447|      23471|    2154|      13447|2020-05-04|2020-05-03|
| 39434| 39433|      26025|    2507|      16810|      24905|    2271|      13447|2020-05-05|2020-05-04|
| 39435| 39434|      27634|    2704|      17781|      26025|    2507|      16810|2020-05-06|2020-05-05|
| 39436| 39435|      29616|    2961|      17781|      27634|    

In [22]:
execute_query("""
              SELECT country, max(confirmed) as max_confirmed
              FROM covid19 a 
              where a.country in ('Mexico', 'Peru', 'Colombia', 'Chile', 'Ecuador', 'Bolivia', 'Venezuela', 'Argentina', 'Brazil', 'Uruguay', 'Paraguay')
              group by country
              order by 2 desc
              """)

+---------+-------------+
|  country|max_confirmed|
+---------+-------------+
|   Brazil|      7675973|
| Colombia|      1642775|
|Argentina|      1625514|
|   Mexico|      1426094|
|     Peru|      1015137|
|    Chile|       608973|
|  Ecuador|       212512|
|  Bolivia|       160124|
|Venezuela|       113558|
| Paraguay|       107932|
|  Uruguay|        19119|
+---------+-------------+



In [23]:
execute_query("""
              SELECT country, confirmed
              FROM covid19 a
              where date = '2020-05-07'
              and country like '%Sal%'
              order by 2 desc
              """)

+-----------+---------+
|    country|confirmed|
+-----------+---------+
|El Salvador|      695|
+-----------+---------+



In [24]:
def its_america(value):
   if value in ['Peru', 'Colombia', 'Chile', 'Ecuador', 'Bolivia', 'Venezuela', 'Argentina', 'Brazil', 'Uruguay', 'Paraguay']:
        return 'SurAmerica'
   elif value in ['Guatemala','El Salvador','Panama', 'Costa Rica', 'Puerto Rico']: 
        return 'CentroAmerica'
   elif value in ['Canada','US','Mexico']: 
        return 'NorteAmerica'
   else: 
        return 'na'

In [25]:
its_america('Mexico')

'NorteAmerica'

In [26]:
from pyspark.sql.functions import udf

udf_its_america = udf(its_america, StringType())
df_with_america = df.withColumn("its_america", udf_its_america(F.col('country')))
df_with_america.show()

+----+----------+---------+------+---------+-----------+-----------+
|item|      date|confirmed|deaths|recovered|    country|its_america|
+----+----------+---------+------+---------+-----------+-----------+
|   0|2020-01-22|        0|     0|        0|Afghanistan|         na|
|   1|2020-01-23|        0|     0|        0|Afghanistan|         na|
|   2|2020-01-24|        0|     0|        0|Afghanistan|         na|
|   3|2020-01-25|        0|     0|        0|Afghanistan|         na|
|   4|2020-01-26|        0|     0|        0|Afghanistan|         na|
|   5|2020-01-27|        0|     0|        0|Afghanistan|         na|
|   6|2020-01-28|        0|     0|        0|Afghanistan|         na|
|   7|2020-01-29|        0|     0|        0|Afghanistan|         na|
|   8|2020-01-30|        0|     0|        0|Afghanistan|         na|
|   9|2020-01-31|        0|     0|        0|Afghanistan|         na|
|  10|2020-02-01|        0|     0|        0|Afghanistan|         na|
|  11|2020-02-02|        0|     0|

In [27]:
df_with_america.createOrReplaceTempView("covid19")

In [28]:
execute_query("show tables")

+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
|        |  covid19|       true|
+--------+---------+-----------+



In [29]:
execute_query("show databases")

+---------+
|namespace|
+---------+
|  default|
+---------+



In [30]:
execute_query("describe covid19")

+-----------+---------+-------+
|   col_name|data_type|comment|
+-----------+---------+-------+
|       item|      int|   null|
|       date|     date|   null|
|  confirmed|      int|   null|
|     deaths|      int|   null|
|  recovered|      int|   null|
|    country|   string|   null|
|its_america|   string|   null|
+-----------+---------+-------+



In [31]:
execute_query("""
              SELECT country, confirmed, its_america
              FROM covid19 a
              where date = '2020-05-07'
              and its_america != 'n/a'
              order by 2 desc
              """)

+--------------+---------+------------+
|       country|confirmed| its_america|
+--------------+---------+------------+
|            US|  1267201|NorteAmerica|
|         Spain|   221447|          na|
|         Italy|   215858|          na|
|United Kingdom|   204387|          na|
|        Russia|   177160|          na|
|        France|   177096|          na|
|       Germany|   169430|          na|
|        Brazil|   135773|  SurAmerica|
|        Turkey|   133721|          na|
|          Iran|   103135|          na|
|         China|    83975|          na|
|        Canada|    66201|NorteAmerica|
|          Peru|    58526|  SurAmerica|
|         India|    56351|          na|
|       Belgium|    51420|          na|
|   Netherlands|    41973|          na|
|  Saudi Arabia|    33731|          na|
|       Ecuador|    30298|  SurAmerica|
|   Switzerland|    30126|          na|
|        Mexico|    29616|NorteAmerica|
+--------------+---------+------------+
only showing top 20 rows



## Bucketing, Sorting and Partitioning

Para la fuente de datos basada en archivos, también es posible agrupar y ordenar o particionar la salida. El agrupamiento y la clasificación solo se aplican a las tablas persistentes:

In [164]:
# spark.sql("drop table covid19_bucketed")

DataFrame[]

In [32]:
df_with_america.write.bucketBy(10, "country").sortBy("confirmed").saveAsTable("covid19_bucketed")

In [33]:
execute_query("""
              SELECT country, confirmed
              FROM covid19_bucketed a
              where date = '2020-05-07'
              order by 2 desc
              """)

+--------------+---------+
|       country|confirmed|
+--------------+---------+
|            US|  1267201|
|         Spain|   221447|
|         Italy|   215858|
|United Kingdom|   204387|
|        Russia|   177160|
|        France|   177096|
|       Germany|   169430|
|        Brazil|   135773|
|        Turkey|   133721|
|          Iran|   103135|
|         China|    83975|
|        Canada|    66201|
|          Peru|    58526|
|         India|    56351|
|       Belgium|    51420|
|   Netherlands|    41973|
|  Saudi Arabia|    33731|
|       Ecuador|    30298|
|   Switzerland|    30126|
|        Mexico|    29616|
+--------------+---------+
only showing top 20 rows



Mientras que la partición se puede usar con save y saveAsTable cuando se usan las API de conjunto de datos.

In [34]:
df_with_america.write.partitionBy("its_america").format("parquet").save("covid19")

In [35]:
df_parquet = spark.read.parquet("./covid19")

In [36]:
df_parquet.describe()

DataFrame[summary: string, item: string, confirmed: string, deaths: string, recovered: string, country: string, its_america: string]

## reference

- Python:  https://github.com/jakevdp/PythonDataScienceHandbook
- Hive: https://github.com/vavasquezhe/apache-hive-course
- SparkSQL: https://spark.apache.org/docs/2.4.0/sql-getting-started.html
- Databricks: https://docs.databricks.com/spark/latest/spark-sql/index.html